<a href="https://colab.research.google.com/github/Bovi-analytics/SupplementalFiles-YongyanChen/blob/main/1_Data_editing_before_fitting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preparation

In [ ]:
from google.colab import drive 
drive.mount('/content/gdrive') 

Mounted at /content/gdrive


In [ ]:
!apt-get update

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:6 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:7 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:8 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:9 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Ign:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:13 https://developer.download.nvidia.co

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.2.0/spark-3.2.0-bin-hadoop3.2.tgz
!tar xf spark-3.2.0-bin-hadoop3.2.tgz
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.0-bin-hadoop3.2"

In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [ ]:
from pyspark.sql.types import LongType, StringType, StructField, StructType, BooleanType, ArrayType, IntegerType, DoubleType, FloatType
import pyspark
from pyspark.sql import functions as F,Window
from pyspark.sql.functions import year, col, array, monotonically_increasing_id, to_json, struct, round, datediff,unix_timestamp,udf,avg
import json
from urllib.request import  urlopen
import pyspark.sql.functions as F

## Load in test-day data

In [ ]:
spark_alldata_formatted= spark \
  .read \
  .parquet('/content/gdrive/Shared drives/Bovi-Analytics/Projects/ChenYongYan/Data/spark_alldata_formatted/')

In [ ]:
spark_alldata_formatted.count()

159173868

In [ ]:
spark_alldata_formatted.printSchema()

root
 |-- AnimalIdentifier: string (nullable = true)
 |-- BirthDate: date (nullable = true)
 |-- Parity: double (nullable = true)
 |-- CalvingDate: date (nullable = true)
 |-- BusinessType: string (nullable = true)
 |-- HerdIdentifier: integer (nullable = true)
 |-- Date: date (nullable = true)
 |-- MilkYieldKg: double (nullable = true)
 |-- FatPercentage: double (nullable = true)
 |-- ProteinPercentage: double (nullable = true)
 |-- SCC: double (nullable = true)
 |-- Urea: double (nullable = true)
 |-- LactosePercentage: double (nullable = true)
 |-- NumberOfMilkings: double (nullable = true)
 |-- StatusProduction: double (nullable = true)
 |-- StatusFictive: double (nullable = true)
 |-- StatusCow: double (nullable = true)
 |-- LactationValue: string (nullable = true)
 |-- DaysInMilk: double (nullable = true)
 |-- TestYear: integer (nullable = true)
 |-- AgeInDays: double (nullable = true)
 |-- FPCM: double (nullable = true)



In [ ]:
spark_alldata_formatted.show(5)

+----------------+----------+------+-----------+------------+--------------+----------+-----------+-------------+-----------------+--------+----+-----------------+----------------+----------------+-------------+---------+--------------+----------+--------+---------+------------------+
|AnimalIdentifier| BirthDate|Parity|CalvingDate|BusinessType|HerdIdentifier|      Date|MilkYieldKg|FatPercentage|ProteinPercentage|     SCC|Urea|LactosePercentage|NumberOfMilkings|StatusProduction|StatusFictive|StatusCow|LactationValue|DaysInMilk|TestYear|AgeInDays|              FPCM|
+----------------+----------+------+-----------+------------+--------------+----------+-----------+-------------+-----------------+--------+----+-----------------+----------------+----------------+-------------+---------+--------------+----------+--------+---------+------------------+
|   AT 0007841167|2013-10-13|   3.0| 2017-09-28|         UBN|       6601503|2017-12-09|       22.2|          3.5|             3.17| 11000.0| 4

# Add filters

### step one _ herd level

herd level has been done in R

In [ ]:
QualifiedHerd= spark \
  .read \
  .option("header", "true")\
  .csv('/content/gdrive/Shared drives/Bovi-Analytics/Projects/ChenYongYan/Data/QualifiedHerd.csv')\
  .drop('_c0')

In [ ]:
QualifiedHerd.printSchema()

root
 |-- HerdIdentifier: integer (nullable = true)
 |-- TestYear: integer (nullable = true)



In [ ]:
QualifiedHerd=QualifiedHerd.\
withColumn('TestYear',QualifiedHerd['TestYear'].cast(IntegerType())).\
withColumn('HerdIdentifier',QualifiedHerd['HerdIdentifier'].cast(IntegerType()))

In [ ]:
QualifiedHerd.show(5)

+--------------+--------+
|HerdIdentifier|TestYear|
+--------------+--------+
|           590|    2007|
|           590|    2008|
|           590|    2009|
|           590|    2010|
|           590|    2011|
+--------------+--------+
only showing top 5 rows



In [ ]:
QualifiedHerd.select(F.countDistinct("HerdIdentifier")).show()

+------------------------------+
|count(DISTINCT HerdIdentifier)|
+------------------------------+
|                         17960|
+------------------------------+



### step two_ cow level

innerjoin with cow data

In [ ]:
spark_alldata_joined=spark_alldata_formatted.join(QualifiedHerd,['HerdIdentifier','TestYear'],'inner')

In [ ]:
spark_alldata_formatted.count()

159173868

In [ ]:
spark_alldata_joined.count()

152504375

In [ ]:
spark_alldata_formatted.select(F.countDistinct("HerdIdentifier")).show()

+------------------------------+
|count(DISTINCT HerdIdentifier)|
+------------------------------+
|                         20760|
+------------------------------+



In [ ]:
spark_alldata_joined.select(F.countDistinct("HerdIdentifier")).show()

+------------------------------+
|count(DISTINCT HerdIdentifier)|
+------------------------------+
|                         17960|
+------------------------------+



#####① Status

In [ ]:
spark_alldata_filtered = spark_alldata_joined . \
  filter((col('ProteinPercentage')!=0.0) & \
      (col('FatPercentage') !=0.0) &\
      (col('StatusCow')==0) &\
      ((col('StatusProduction')==1) |(col('StatusProduction')==0 ))
  )

In [ ]:
spark_alldata_filtered.agg(F.countDistinct("HerdIdentifier"),F.countDistinct("AnimalIdentifier")).collect()

[Row(count(DISTINCT HerdIdentifier)=17582, count(DISTINCT AnimalIdentifier)=5962324)]

In [ ]:
spark_alldata_filtered.count()

143883847

##### ② MY,Pro,Fat

Calculate quantile

In [ ]:
spark_alldata_filtered.show(5)

+--------------+--------+----------------+----------+------+-----------+------------+----------+-----------+-------------+-----------------+--------+----+-----------------+----------------+----------------+-------------+---------+--------------+----------+---------+------------------+
|HerdIdentifier|TestYear|AnimalIdentifier| BirthDate|Parity|CalvingDate|BusinessType|      Date|MilkYieldKg|FatPercentage|ProteinPercentage|     SCC|Urea|LactosePercentage|NumberOfMilkings|StatusProduction|StatusFictive|StatusCow|LactationValue|DaysInMilk|AgeInDays|              FPCM|
+--------------+--------+----------------+----------+------+-----------+------------+----------+-----------+-------------+-----------------+--------+----+-----------------+----------------+----------------+-------------+---------+--------------+----------+---------+------------------+
|       6601503|    2017| AT 0007841167  |2013-10-13|   3.0| 2017-09-28|         UBN|2017-12-09|       22.2|          3.5|             3.17| 1

In [ ]:
QuantilesMilkYield = spark_alldata_filtered.agg(F.expr('percentile(MilkYieldKg,array(0.01,0.99))').alias('Q1')).select('Q1').collect()[0][0]
QuantilesProteinPercentage = spark_alldata_filtered.agg(F.expr('percentile(ProteinPercentage,array(0.01,0.99))').alias('Q1')).select('Q1').collect()[0][0]
QuantilesFatPercentage = spark_alldata_filtered.agg(F.expr('percentile(FatPercentage,array(0.01,0.99))').alias('Q1')).select('Q1').collect()[0][0]

In [ ]:
QuantilesMilkYield 

[8.5, 50.0]

In [ ]:
QuantilesProteinPercentage

[2.8, 4.81]

In [ ]:
QuantilesFatPercentage

[2.93, 6.53]

In [ ]:
spark_alldata_filtered = spark_alldata_filtered. \
  filter((col('MilkYieldKg')<QuantilesMilkYield[1]) &\
      (col('ProteinPercentage')<QuantilesProteinPercentage[1]) &\
      (col('ProteinPercentage')>QuantilesProteinPercentage[0]) &\
      (col('FatPercentage')<QuantilesFatPercentage[1]) &\
      (col('FatPercentage')>QuantilesFatPercentage[0])  
  )

In [ ]:
spark_alldata_filtered.agg(F.countDistinct("HerdIdentifier"),F.countDistinct("AnimalIdentifier")).collect()

[Row(count(DISTINCT HerdIdentifier)=17582, count(DISTINCT AnimalIdentifier)=5940347)]

In [ ]:
spark_alldata_filtered.count()

137166910

#####③Parity

In [ ]:
spark_alldata_filtered = spark_alldata_filtered.\
  filter((col('AgeInDays')/365 > col('Parity') ))

In [ ]:
spark_alldata_filtered.count()

137165667

In [ ]:
spark_alldata_filtered.agg(F.countDistinct("HerdIdentifier"),F.countDistinct("AnimalIdentifier")).collect()

[Row(count(DISTINCT HerdIdentifier)=17582, count(DISTINCT AnimalIdentifier)=5940311)]

Quantile AgeInDays per parity

List comprehension

In [ ]:
QuantilesAgeInDays=[(i, spark_alldata_filtered.filter((col('Parity')==i)).agg(F.expr('percentile(AgeInDays,array(0.01,0.99))').alias('Q1')).select('Q1').collect()[0][0]) for i in range(1,21)]

In [ ]:
QuantilesAgeInDays

[(1, [723.0, 1441.0]),
 (2, [1084.0, 1932.0]),
 (3, [1447.0, 2421.0]),
 (4, [1810.0, 2895.0]),
 (5, [2174.0, 3361.0]),
 (6, [2538.0, 3820.0]),
 (7, [2903.0, 4279.0]),
 (8, [3268.0, 4742.0]),
 (9, [3630.0, 5197.260000000009]),
 (10, [3993.0, 5639.0]),
 (11, [4354.0, 6091.0]),
 (12, [4711.0, 6480.129999999997]),
 (13, [5066.99, 6957.0]),
 (14, [5431.780000000001, 7349.219999999999]),
 (15, [5760.599999999999, 7305.64]),
 (16, [6051.52, 8232.599999999999]),
 (17, [6593.73, 7858.12]),
 (18, [7034.679999999999, 7386.32]),
 (19, None),
 (20, None)]

In [ ]:
QuantilesAgeInDays[1][1]

[1084.0, 1932.0]

In [ ]:
QuantilesAgeInDays[1][1][1]

1932.0

In [ ]:
spark_alldata_filtered = spark_alldata_filtered.\
  filter(((col('Parity')==1)&(col('AgeInDays')>QuantilesAgeInDays[0][1][0])&(col('AgeInDays')<QuantilesAgeInDays[0][1][1]))|\
     ((col('Parity')==2)&(col('AgeInDays')>QuantilesAgeInDays[1][1][0])&(col('AgeInDays')<QuantilesAgeInDays[1][1][1]))|\
     ((col('Parity')==3)&(col('AgeInDays')>QuantilesAgeInDays[2][1][0])&(col('AgeInDays')<QuantilesAgeInDays[2][1][1]))|\
     ((col('Parity')==4)&(col('AgeInDays')>QuantilesAgeInDays[3][1][0])&(col('AgeInDays')<QuantilesAgeInDays[3][1][1]))|\
     ((col('Parity')==5)&(col('AgeInDays')>QuantilesAgeInDays[4][1][0])&(col('AgeInDays')<QuantilesAgeInDays[4][1][1]))|\
     ((col('Parity')==6)&(col('AgeInDays')>QuantilesAgeInDays[5][1][0])&(col('AgeInDays')<QuantilesAgeInDays[5][1][1]))|\
      ((col('Parity')==7)&(col('AgeInDays')>QuantilesAgeInDays[6][1][0])&(col('AgeInDays')<QuantilesAgeInDays[6][1][1]))|\
       ((col('Parity')==8)&(col('AgeInDays')>QuantilesAgeInDays[7][1][0])&(col('AgeInDays')<QuantilesAgeInDays[7][1][1]))|\
        ((col('Parity')==9)&(col('AgeInDays')>QuantilesAgeInDays[8][1][0])&(col('AgeInDays')<QuantilesAgeInDays[8][1][1]))|\
         ((col('Parity')==10)&(col('AgeInDays')>QuantilesAgeInDays[9][1][0])&(col('AgeInDays')<QuantilesAgeInDays[9][1][1]))|\
          ((col('Parity')==11)&(col('AgeInDays')>QuantilesAgeInDays[10][1][0])&(col('AgeInDays')<QuantilesAgeInDays[10][1][1]))|\
           ((col('Parity')==12)&(col('AgeInDays')>QuantilesAgeInDays[11][1][0])&(col('AgeInDays')<QuantilesAgeInDays[11][1][1]))|\
            ((col('Parity')==13)&(col('AgeInDays')>QuantilesAgeInDays[12][1][0])&(col('AgeInDays')<QuantilesAgeInDays[12][1][1]))|\
             ((col('Parity')==14)&(col('AgeInDays')>QuantilesAgeInDays[13][1][0])&(col('AgeInDays')<QuantilesAgeInDays[13][1][1]))|\
              ((col('Parity')==15)&(col('AgeInDays')>QuantilesAgeInDays[14][1][0])&(col('AgeInDays')<QuantilesAgeInDays[14][1][1]))|\
               ((col('Parity')==16)&(col('AgeInDays')>QuantilesAgeInDays[15][1][0])&(col('AgeInDays')<QuantilesAgeInDays[15][1][1]))|\
                ((col('Parity')==17)&(col('AgeInDays')>QuantilesAgeInDays[16][1][0])&(col('AgeInDays')<QuantilesAgeInDays[16][1][1]))|\
                 ((col('Parity')==18)&(col('AgeInDays')>QuantilesAgeInDays[17][1][0])&(col('AgeInDays')<QuantilesAgeInDays[17][1][1]))
               
  )

In [ ]:
spark_alldata_filtered.agg(F.countDistinct("HerdIdentifier"),F.countDistinct("AnimalIdentifier")).collect()

[Row(count(DISTINCT HerdIdentifier)=17580, count(DISTINCT AnimalIdentifier)=5900893)]

In [ ]:
spark_alldata_filtered.count()

134096231

### step three_lactation level

#### DaysInMilk

In [ ]:
QuantilesDaysInMilk = spark_alldata_filtered.agg(F.expr('percentile(DaysInMilk,array(0.01,0.99))').alias('Q1')).select('Q1').collect()[0][0]

In [ ]:
spark_alldata_filtered = spark_alldata_filtered. \
  filter((col('DaysInMilk')<QuantilesDaysInMilk[1]))

In [ ]:
spark_alldata_filtered.agg(F.countDistinct("HerdIdentifier"),F.countDistinct("AnimalIdentifier")).collect()

[Row(count(DISTINCT HerdIdentifier)=17580, count(DISTINCT AnimalIdentifier)=5894719)]

In [ ]:
spark_alldata_filtered.count()

132737234

#### CalvingInterval

Keep the last record for each lactation and then calculate calving interval

In [ ]:
dfc=spark_alldata_filtered.select('AnimalIdentifier','Parity','CalvingDate').drop_duplicates()

import pyspark.sql.functions as F
from pyspark.sql import Window

w=Window.partitionBy('AnimalIdentifier').\
  orderBy(F.asc('Parity'))

dfc1=dfc.withColumn('CalvingInterval',\
  F.datediff(dfc.CalvingDate,F.lag(dfc.CalvingDate,1).over(w)))

In [ ]:
dfc1.show()

+----------------+------+-----------+---------------+
|AnimalIdentifier|Parity|CalvingDate|CalvingInterval|
+----------------+------+-----------+---------------+
|   AT 5378120161|   2.0| 2017-02-23|           null|
|   AT 5378120161|   3.0| 2018-05-08|            439|
|   AT 5378120161|   4.0| 2019-04-10|            337|
|   AT 9200602075|   2.0| 2013-04-17|           null|
|   AT 9200602075|   3.0| 2014-08-29|            499|
|   AT 9200602075|   4.0| 2015-07-26|            331|
|   BE 1136242476|   1.0| 2019-10-16|           null|
|   BE 4712646635|   1.0| 2007-08-29|           null|
|   BE 4712646635|   2.0| 2008-10-07|            405|
|   BE 4712646635|   3.0| 2010-01-03|            453|
|   BE 4712646635|   4.0| 2011-03-12|            433|
|   BE 4712646635|   5.0| 2012-04-27|            412|
|   BE 4712646635|   6.0| 2013-07-25|            454|
|   BE 4712646635|   7.0| 2016-01-02|            891|
|   BE 4712646635|   8.0| 2017-05-24|            508|
|   BE 4712646635|   9.0| 20

In [ ]:
QuantilesCalvingInterval = dfc1.agg(F.expr('percentile(CalvingInterval,array(0.01,0.99))').alias('Q1')).select('Q1').collect()[0][0]

In [ ]:
QuantilesCalvingInterval

[314.0, 695.0]

In [ ]:
dfc1.count()

16772035

first merge then quantile

In [ ]:
spark_alldata_filtered=spark_alldata_filtered.join(dfc1,['AnimalIdentifier','Parity','CalvingDate'],'inner')

In [ ]:
QuantilesCalvingInterval = spark_alldata_filtered.agg(F.expr('percentile(CalvingInterval,array(0.01,0.99))').alias('Q1')).select('Q1').collect()[0][0]

In [ ]:
QuantilesCalvingInterval

[316.0, 683.0]

In [ ]:
spark_alldata_filtered=spark_alldata_filtered.filter((col('CalvingInterval')<QuantilesCalvingInterval[1])&\
      (col('CalvingInterval')>QuantilesCalvingInterval[0])|\
      col('CalvingInterval').isNull())

In [ ]:
spark_alldata_filtered.agg(F.countDistinct("HerdIdentifier"),F.countDistinct("AnimalIdentifier")).collect()

[Row(count(DISTINCT HerdIdentifier)=17580, count(DISTINCT AnimalIdentifier)=5894719)]

In [ ]:
spark_alldata_filtered.count()

130939857

In [ ]:
spark_alldata_filtered.show()

+----------------+------+-----------+--------------+--------+----------+------------+----------+-----------+-------------+-----------------+--------+----+-----------------+----------------+----------------+-------------+---------+--------------+----------+---------+------------------+---------------+
|AnimalIdentifier|Parity|CalvingDate|HerdIdentifier|TestYear| BirthDate|BusinessType|      Date|MilkYieldKg|FatPercentage|ProteinPercentage|     SCC|Urea|LactosePercentage|NumberOfMilkings|StatusProduction|StatusFictive|StatusCow|LactationValue|DaysInMilk|AgeInDays|              FPCM|CalvingInterval|
+----------------+------+-----------+--------------+--------+----------+------------+----------+-----------+-------------+-----------------+--------+----+-----------------+----------------+----------------+-------------+---------+--------------+----------+---------+------------------+---------------+
| AT 0107399618  |   2.0| 2008-11-20|       1552346|    2008|2005-09-26|         UBN|2008-12-0

In [ ]:
spark_alldata_filtered.describe().show()

In [ ]:
spark_alldata_filtered.\
  repartition(1).\
  write.\
  parquet(path='/content/gdrive/Shared drives/Bovi-Analytics/Projects/ChenYongYan/Data/spark_alldata_filtered/',mode="overwrite")